In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

In [3]:
ur_files = ddf.read_csv('daily_temp_county_level/daily_temp_county_level_2009.csv', dtype = {'fips': str})
temp = ur_files.compute()
temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
temp.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2009-01-01,41.323529,34.311765,49.282353,2009.0
1,01001,2009-01-02,51.400000,44.520000,61.340000,2009.0
2,01001,2009-01-03,65.670588,58.135294,72.923529,2009.0
3,01001,2009-01-04,64.529412,59.023529,69.358824,2009.0
4,01001,2009-01-05,63.382353,60.852941,66.270588,2009.0


In [4]:
ur_files = ddf.read_csv('average_max_temp_6190.csv', dtype = {'fips': str})
average_max = ur_files.compute()
average_max = average_max.loc[:, ~average_max.columns.str.contains('^Unnamed')]
average_max.head()

,fips,YEAR,average_temp,average_min_temp,average_max_temp
0,01001,1975.5,88.086863,78.430588,99.182549
1,01003,1975.5,86.279225,80.531628,94.950310
2,01005,1975.5,85.219630,75.624444,97.295556
3,01007,1975.5,86.538333,77.821250,97.962917
4,01009,1975.5,83.823333,74.926042,96.336875


In [5]:
average_max.shape

(2839, 5)

In [6]:
def findList(fips, average_max):
    average_max_list = []
    average_max_list.append(fips)
    average_max_list.append(average_max)
    return average_max_list

In [7]:
average_max['average_max_list'] = average_max.apply(lambda x: findList(x['fips'], x['average_max_temp']), axis=1)
average_max.head()

,fips,YEAR,average_temp,average_min_temp,average_max_temp,average_max_list
0,01001,1975.5,88.086863,78.430588,99.182549,"[01001, 99.18254901960783]"
1,01003,1975.5,86.279225,80.531628,94.950310,"[01003, 94.95031007751938]"
2,01005,1975.5,85.219630,75.624444,97.295556,"[01005, 97.29555555555557]"
3,01007,1975.5,86.538333,77.821250,97.962917,"[01007, 97.96291666666666]"
4,01009,1975.5,83.823333,74.926042,96.336875,"[01009, 96.336875]"


In [8]:
average_max_list = average_max['average_max_list'].tolist()
average_max_list[:5]

[['01001', 99.18254901960783],
 ['01003', 94.95031007751938],
 ['01005', 97.29555555555557],
 ['01007', 97.96291666666666],
 ['01009', 96.336875]]

In [9]:
df_example = temp[temp['fips'] == '01001']
df_example.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2009-01-01,41.323529,34.311765,49.282353,2009.0
1,01001,2009-01-02,51.400000,44.520000,61.340000,2009.0
2,01001,2009-01-03,65.670588,58.135294,72.923529,2009.0
3,01001,2009-01-04,64.529412,59.023529,69.358824,2009.0
4,01001,2009-01-05,63.382353,60.852941,66.270588,2009.0


In [10]:
df_example

,fips,date,temp,min_temp,max_temp,year
0,01001,2009-01-01,41.323529,34.311765,49.282353,2009.0
1,01001,2009-01-02,51.400000,44.520000,61.340000,2009.0
2,01001,2009-01-03,65.670588,58.135294,72.923529,2009.0
3,01001,2009-01-04,64.529412,59.023529,69.358824,2009.0
4,01001,2009-01-05,63.382353,60.852941,66.270588,2009.0
...,...,...,...,...,...,...
360,01001,2009-12-27,39.058824,28.876471,53.088235,2009.0
361,01001,2009-12-28,38.017647,30.370588,45.847059,2009.0
362,01001,2009-12-29,37.123529,27.552941,50.717647,2009.0
363,01001,2009-12-30,42.135294,32.911765,49.964706,2009.0


In [11]:
N = 5
c1 = df_example['max_temp'].ge(100)
g = (c1 != c1.shift()).cumsum()
df1 = df_example.assign(
    cnt=df_example.groupby(g).date.transform('count')
  , n=df_example.groupby(g).agg('cumcount')
  , g=g
)
df1.head()

,fips,date,temp,min_temp,max_temp,year,cnt,n,g
0,01001,2009-01-01,41.323529,34.311765,49.282353,2009.0,365,0,1
1,01001,2009-01-02,51.400000,44.520000,61.340000,2009.0,365,1,1
2,01001,2009-01-03,65.670588,58.135294,72.923529,2009.0,365,2,1
3,01001,2009-01-04,64.529412,59.023529,69.358824,2009.0,365,3,1
4,01001,2009-01-05,63.382353,60.852941,66.270588,2009.0,365,4,1


In [12]:
c2 = df1.cnt.ge(N)
c3 = df1.n.mod(N).eq(0) & df1.n.le(df1.cnt-N)
df_example['flag'] = np.where(c1 & c2 & c3, 1, 0)
df_example

/global/homes/w/wenhuan/.conda/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,fips,date,temp,min_temp,max_temp,year,flag
0,01001,2009-01-01,41.323529,34.311765,49.282353,2009.0,0
1,01001,2009-01-02,51.400000,44.520000,61.340000,2009.0,0
2,01001,2009-01-03,65.670588,58.135294,72.923529,2009.0,0
3,01001,2009-01-04,64.529412,59.023529,69.358824,2009.0,0
4,01001,2009-01-05,63.382353,60.852941,66.270588,2009.0,0
...,...,...,...,...,...,...,...
360,01001,2009-12-27,39.058824,28.876471,53.088235,2009.0,0
361,01001,2009-12-28,38.017647,30.370588,45.847059,2009.0,0
362,01001,2009-12-29,37.123529,27.552941,50.717647,2009.0,0
363,01001,2009-12-30,42.135294,32.911765,49.964706,2009.0,0


In [13]:
df_example['flag'].unique()

array([0])

In [14]:
def findHeatwaveCount(temp, fips, average_max):
    df = temp[temp['fips'] == fips]
    c1 = df['max_temp'].ge(average_max)
    N = 5
    g = (c1 != c1.shift()).cumsum()
    df1 = df.assign(
        cnt=df.groupby(g).date.transform('count')
      , n=df.groupby(g).agg('cumcount')
      , g=g
    )
    c2 = df1.cnt.ge(N)
    c3 = df1.n.mod(N).eq(0) & df1.n.le(df1.cnt-N)
    df['flag'] = np.where(c1 & c2 & c3, 1, 0)
    del(df1)
    return df

In [15]:
frames = []
for item in average_max_list:
    fips = item[0]
    average_number = item[1]
    df = findHeatwaveCount(temp, fips, average_number)
    frames.append(df)

/global/homes/w/wenhuan/.conda/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [16]:
heatwave = pd.concat(frames)
heatwave.head()

,fips,date,temp,min_temp,max_temp,year,flag
0,01001,2009-01-01,41.323529,34.311765,49.282353,2009.0,0
1,01001,2009-01-02,51.400000,44.520000,61.340000,2009.0,0
2,01001,2009-01-03,65.670588,58.135294,72.923529,2009.0,0
3,01001,2009-01-04,64.529412,59.023529,69.358824,2009.0,0
4,01001,2009-01-05,63.382353,60.852941,66.270588,2009.0,0


In [17]:
heatwave['flag'].unique()

array([0, 1])

In [18]:
heatwave_count = heatwave.groupby(['fips']).sum()
heatwave_count = heatwave_count.reset_index()
heatwave_count.head()

,fips,temp,min_temp,max_temp,year,flag
0,01001,23384.934118,20139.449412,27493.194118,733285.0,0
1,01003,24770.905892,21965.415263,27944.841005,733285.0,1
2,01005,23625.330556,20117.434722,28126.768056,733285.0,2
3,01007,22696.385714,19621.642857,26758.667857,733285.0,0
4,01009,22230.142992,19122.953977,26126.274053,733285.0,0


In [19]:
heatwave_count['year'] = 2009
heatwave_count = heatwave_count.rename(columns = {'flag': 'heatwave_count'})
heatwave_count = heatwave_count[['fips', 'heatwave_count', 'year']]
heatwave_count.head()

,fips,heatwave_count,year
0,01001,0,2009
1,01003,1,2009
2,01005,2,2009
3,01007,0,2009
4,01009,0,2009


In [20]:
heatwave_count.shape

(2751, 3)

In [21]:
heatwave_count['heatwave_count'].max()

28

In [22]:
example = heatwave_count[heatwave_count['heatwave_count'] == 28]
example

,fips,heatwave_count,year
319,12095,28,2009


In [23]:
df_max = average_max[average_max['fips'] == '12095']
df_max

,fips,YEAR,average_temp,average_min_temp,average_max_temp,average_max_list
323,12095,1975.5,75.592472,68.739276,85.498989,"[12095, 85.49898876404494]"


In [24]:
df_temp = temp[temp['fips'] == '12095']
df_temp

,fips,date,temp,min_temp,max_temp,year
128543,12095,2009-01-01,60.062547,51.850187,69.219850,2009.0
128544,12095,2009-01-02,63.158427,54.461423,74.049064,2009.0
128545,12095,2009-01-03,64.483895,52.910112,75.174157,2009.0
128546,12095,2009-01-04,68.285393,58.388764,78.662547,2009.0
128547,12095,2009-01-05,67.280899,55.186517,78.289139,2009.0
...,...,...,...,...,...,...
128903,12095,2009-12-27,52.896296,49.967078,57.554321,2009.0
128904,12095,2009-12-28,56.644444,49.030453,67.232510,2009.0
128905,12095,2009-12-29,48.031276,38.760905,62.381481,2009.0
128906,12095,2009-12-30,55.120576,43.695062,73.124280,2009.0


In [25]:
df_count = findHeatwaveCount(df_temp, '12095', 85.5)
df_count

,fips,date,temp,min_temp,max_temp,year,flag
128543,12095,2009-01-01,60.062547,51.850187,69.219850,2009.0,0
128544,12095,2009-01-02,63.158427,54.461423,74.049064,2009.0,0
128545,12095,2009-01-03,64.483895,52.910112,75.174157,2009.0,0
128546,12095,2009-01-04,68.285393,58.388764,78.662547,2009.0,0
128547,12095,2009-01-05,67.280899,55.186517,78.289139,2009.0,0
...,...,...,...,...,...,...,...
128903,12095,2009-12-27,52.896296,49.967078,57.554321,2009.0,0
128904,12095,2009-12-28,56.644444,49.030453,67.232510,2009.0,0
128905,12095,2009-12-29,48.031276,38.760905,62.381481,2009.0,0
128906,12095,2009-12-30,55.120576,43.695062,73.124280,2009.0,0


In [26]:
df_50 = df_count.iloc[:50]
df_50

,fips,date,temp,min_temp,max_temp,year,flag
128543,12095,2009-01-01,60.062547,51.850187,69.219850,2009.0,0
128544,12095,2009-01-02,63.158427,54.461423,74.049064,2009.0,0
128545,12095,2009-01-03,64.483895,52.910112,75.174157,2009.0,0
128546,12095,2009-01-04,68.285393,58.388764,78.662547,2009.0,0
128547,12095,2009-01-05,67.280899,55.186517,78.289139,2009.0,0
128548,12095,2009-01-06,69.155849,60.992830,81.129057,2009.0,0
128549,12095,2009-01-07,68.546442,60.486891,78.923970,2009.0,0
128550,12095,2009-01-08,59.155431,45.380150,72.718352,2009.0,0
128551,12095,2009-01-09,60.279401,47.727341,71.081273,2009.0,0
128552,12095,2009-01-10,61.141948,48.523970,75.852434,2009.0,0


In [27]:
df_100 = df_count.iloc[50:100]
df_100

,fips,date,temp,min_temp,max_temp,year,flag
128593,12095,2009-02-20,56.661049,47.144569,68.872659,2009.0,0
128594,12095,2009-02-21,54.163670,39.466292,70.111236,2009.0,0
128595,12095,2009-02-22,62.456929,50.408989,75.665918,2009.0,0
128596,12095,2009-02-23,60.780524,51.067416,71.630712,2009.0,0
128597,12095,2009-02-24,56.476030,46.201873,70.272659,2009.0,0
128598,12095,2009-02-25,60.845693,49.411985,72.985019,2009.0,0
128599,12095,2009-02-26,62.086891,50.228090,74.984270,2009.0,0
128600,12095,2009-02-27,64.557678,51.896255,81.667416,2009.0,0
128601,12095,2009-02-28,67.923221,54.482022,81.864045,2009.0,0
128602,12095,2009-03-01,63.171161,49.847566,77.258801,2009.0,0


In [28]:
df_150 = df_count.iloc[100:150]
df_150

,fips,date,temp,min_temp,max_temp,year,flag
128643,12095,2009-04-11,74.657303,65.917228,84.429588,2009.0,0
128644,12095,2009-04-12,74.256929,63.823596,84.977528,2009.0,0
128645,12095,2009-04-13,72.511610,63.419101,84.278652,2009.0,0
128646,12095,2009-04-14,68.689513,63.035206,76.665543,2009.0,0
128647,12095,2009-04-15,69.456929,59.021723,81.953558,2009.0,0
128648,12095,2009-04-16,68.083146,56.476779,81.958052,2009.0,0
128649,12095,2009-04-17,69.271536,60.483895,77.645318,2009.0,0
128650,12095,2009-04-18,68.510112,58.173783,79.931835,2009.0,0
128651,12095,2009-04-19,69.366667,56.208989,82.814981,2009.0,0
128652,12095,2009-04-20,72.447566,63.473034,82.315356,2009.0,0


In [29]:
df_200 = df_count.iloc[150:200]
df_200

,fips,date,temp,min_temp,max_temp,year,flag
128693,12095,2009-05-31,78.704869,66.849438,88.273783,2009.0,0
128694,12095,2009-06-01,79.272659,68.349064,89.138577,2009.0,0
128695,12095,2009-06-02,79.103745,70.253184,88.973034,2009.0,0
128696,12095,2009-06-03,78.940075,71.804869,87.983895,2009.0,0
128697,12095,2009-06-04,76.816105,70.247940,85.548315,2009.0,0
128698,12095,2009-06-05,76.540075,71.158427,88.132584,2009.0,0
128699,12095,2009-06-06,76.402247,67.947191,84.613483,2009.0,0
128700,12095,2009-06-07,75.583895,70.988764,84.232959,2009.0,0
128701,12095,2009-06-08,77.249064,69.162172,88.797004,2009.0,1
128702,12095,2009-06-09,79.397753,71.176779,90.129588,2009.0,0


In [30]:
df_250 = df_count.iloc[200:250]
df_250

,fips,date,temp,min_temp,max_temp,year,flag
128743,12095,2009-07-20,76.519476,71.176779,86.168165,2009.0,0
128744,12095,2009-07-21,80.961798,72.568165,90.860674,2009.0,0
128745,12095,2009-07-22,82.336330,73.579026,92.819101,2009.0,1
128746,12095,2009-07-23,82.834831,75.383895,93.510861,2009.0,0
128747,12095,2009-07-24,82.817228,74.995131,92.191011,2009.0,0
128748,12095,2009-07-25,82.660300,75.024345,93.008240,2009.0,0
128749,12095,2009-07-26,79.211985,74.229963,90.440824,2009.0,0
128750,12095,2009-07-27,79.320599,73.572659,89.241948,2009.0,1
128751,12095,2009-07-28,83.050187,74.414232,92.659551,2009.0,0
128752,12095,2009-07-29,84.046067,75.925843,94.464045,2009.0,0


In [32]:
heatwave_count.to_csv('heatwave_count_2009.csv')